In [61]:
import os
import pandas as pd
import numpy as np
import PIL
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import load_model,save_model
from keras.callbacks import ModelCheckpoint
from matplotlib.patches import Rectangle
import os
from scipy.misc import imsave

import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Input, Lambda
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from math import sqrt
from keras.callbacks import History 
import gc
from numpy.random import randint, choice

import keras.backend as K

In [10]:
from keras.preprocessing import image

def read_img(filepath, size):
    img = image.load_img((filepath), target_size=size)
    img = image.img_to_array(img,data_format='channels_last')
    return img

In [66]:
input_dim = (45,45,3)

basePath = "/home/rsk/Documents/Projects/GenericModels/SiameseNetworks/Dataset"
trainPath = os.path.join(basePath,'Train')
testPath = os.path.join(basePath,'Test')

In [67]:
superList = []

os.chdir(trainPath)

folderList = os.listdir("./")

for folder in folderList:
    os.chdir(os.path.join(trainPath,folder))
    
    
    imgList = os.listdir("./")
    
    #Appending positive samples
    for i in range(14):
        img1 = read_img(os.path.join(trainPath,folder,imgList[i]), (45,45))
        img2 = read_img(os.path.join(trainPath,folder,imgList[i+1]), (45,45))
        
        superList.append(((img1,img2),1))
        
    img1 = read_img(os.path.join(trainPath,folder,imgList[14]), (45,45))
    img2 = read_img(os.path.join(trainPath,folder,imgList[0]), (45,45))
    
    superList.append(((img1,img2),1))
    
    #Appending negative samples
    
    for i in range(14):
        
        altFolder = choice(folderList)
        while(altFolder==folder):
            altFolder = choice(folderList)
            
        altImage = choice(os.listdir(os.path.join(trainPath,altFolder)))
            
        img1 = read_img(os.path.join(trainPath,folder,imgList[i]), (45,45))
        img2 = read_img(os.path.join(trainPath,altFolder,altImage), (45,45))
        
        superList.append(((img1,img2),0))
        
    

In [68]:
np.random.shuffle(superList)

In [ ]:
i=34

plt.subplot(211)
plt.imshow(superList[i][0][0])
    
plt.subplot(212)
plt.imshow(superList[i][0][1])

print(superList[i][1])

In [69]:
baseModel = Sequential()
baseModel.add(BatchNormalization(input_shape = input_dim))
baseModel.add(Conv2D(64, (3,3),activation='relu', data_format='channels_last'))

baseModel.add(Dropout(0.3))
baseModel.add(Conv2D(32, (3,3),activation='relu'))
baseModel.add(Flatten())
baseModel.add(Dense(256, activation='relu'))
baseModel.add(Dense(32, activation='relu'))


In [70]:
inputA = Input(shape = input_dim)
inputB = Input(shape = input_dim)

outputA = baseModel(inputA)
outputB = baseModel(inputB)

In [74]:
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([outputA, outputB])
prediction = Dense(1,activation='sigmoid')(L1_distance)
siamese_net = Model(inputs=[inputA,inputB],outputs=prediction)

siamese_net.compile(loss="binary_crossentropy",optimizer='adadelta',metrics=['accuracy'])

In [72]:
#Extracting data
inputAimgs = np.array([x[0][0] for x in superList])
inputBimgs = np.array([x[0][1] for x in superList])
target = np.array([x[1] for x in superList])

In [75]:
siamese_net.fit([inputAimgs,inputBimgs], target, batch_size = 20,epochs = 15 )

Epoch 1/15
870/870 [==============================] - 3s 4ms/step - loss: 0.7167 - acc: 0.5437
Epoch 2/15
870/870 [==============================] - 3s 3ms/step - loss: 0.5766 - acc: 0.6954
Epoch 3/15
870/870 [==============================] - 3s 3ms/step - loss: 0.5153 - acc: 0.7471
Epoch 4/15
870/870 [==============================] - 3s 3ms/step - loss: 0.3491 - acc: 0.8437
Epoch 5/15
870/870 [==============================] - 3s 3ms/step - loss: 0.2516 - acc: 0.8874
Epoch 6/15
870/870 [==============================] - 3s 3ms/step - loss: 0.2452 - acc: 0.9000
Epoch 7/15
870/870 [==============================] - 3s 3ms/step - loss: 0.1088 - acc: 0.9632
Epoch 8/15
870/870 [==============================] - 3s 3ms/step - loss: 0.0727 - acc: 0.9736
Epoch 9/15
870/870 [==============================] - 3s 3ms/step - loss: 0.0522 - acc: 0.9816
Epoch 10/15
870/870 [==============================] - 3s 3ms/step - loss: 0.0090 - acc: 0.9977
Epoch 11/15
870/870 [============================

In [64]:
help(siamese_net.fit)

Help on method fit in module keras.engine.training:

fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, **kwargs) method of keras.engine.training.Model instance
    Trains the model for a fixed number of epochs (iterations on a dataset).
    
    # Arguments
        x: Numpy array of training data (if the model has a single input),
            or list of Numpy arrays (if the model has multiple inputs).
            If input layers in the model are named, you can also pass a
            dictionary mapping input names to Numpy arrays.
            `x` can be `None` (default) if feeding from
            framework-native tensors (e.g. TensorFlow data tensors).
        y: Numpy array of target (label) data
            (if the model has a single output),
            or list of Numpy arrays (if the model has m